* 제한시간: 24시간



```Customer```, ```Order``` 컨트랙을 개발하세요.

컨트랙을 개발하면서 필요한 상태변수와 함수를 구성하고, 그 명칭과 입출력과 수식어도 의미있게 결정한다.



```Customer``` 컨트랙은 별도로 배포하면, ```Order``` 컨트랙을 통해서 데이터를 입력하지 않을 수 있지만, 이 문제에서는 ```new``` 명령어로 ```Customer``` 객체를 만들어 사용하자.

함수를 호출할 때 gas는 넉넉히 배정하도록 하자 (REMIX에서 테스트하면서 콘솔에서 출력되는 gas를 확인).

주문을 삭제하는 기능은 구현하지 않는다.

또한 온라인 주문에서와 같이 주문자는 여러 명이 있을 수 있다는 점에서 주소키를 1, 2, 3 사용하고,

이런 주소키별로 복수의 주문은 가능하지 않도록 구현한다. 현재는 1:1 관계, 즉 주소키별로 주문은 1건만 가능하다.



### ```Customer``` 컨트랙의 멤버 속성과 함수:

- 상태변수

	- 고객ID, 이름, 전화번호, 주소, 마일리지, 고객여부 (고객이면 true)

    - 주소키에 대한 ```mapping``` 등 그 외 필요한 상태변수를 추가할 수 있다.

- 멤버함수

	- 마일리지 추가 함수: 주문금액의 1%를 마일리지로 부여

	- 고객정보 입력 함수: 고객정보 입력은 고객ID가 같으면 입력이 되지 않게 한다.

	- 그 외 필요한 함수를 추가할 수 있다.



### ```Order``` 컨트랙의 멤버 속성과 함수:

- 상태변수

	- 주문ID, 상품명, 개수, 금액, 시간, 상태, 배송지, 주문여부 (주문하면 true)

	- 주소키에 대한 ```mapping``` 등 그 외 필요한 상태변수를 추가할 수 있다.

- 멤버함수

	- 주문 함수

        - 위 상태변수를 설정한다. 주문을 받으면서 상태는 "Ordered"로, 시간은 현재시간으로 설정한다.

        - 주문 금액 반드시 입금되어야 한다.

        - 마일리지는 ```Customer``` 컨트랙을 통하여 설정 (주문금액의 1%)

        - bidirectional map을 설정하여 주문ID를 통하여 주문자 주소키를 알 수 있도록 힌다.

	- 고객정보 입력 함수: 고객정보를 입력한다 (```Customer``` 컨트랙을 통해 설정)

	- 주문내역 출력 함수: 주문했던 고객의 주소키별로 주문ID, 상품명, 상태, 배송지 출력

	- 주문ID로 주문내역 조회 함수: 주문ID를 입력하면 주문ID, 상품명, 상태, 배송지 출력

	- 환불처리 함수: 상품ID에 대해 주문상태를 "refunded"로, 주문금액도 0으로 설정하고 그에 따라 잔고도 감해준다.

    - 그 외 필요한 함수를 추가할 수 있다.



### 사용사례



아래 사용사례 항목별 10점씩 100점, geth로 하면 가점 10점이 배점된다.



#### 1-1 주소 및 잔고 출력

주소는 3개 미리 만들어져 있어야 한다. 주소1, 주소2, 주소3과 각 잔고를 출력한다. 현재의 블록번호도 출력한다.



#### 1-2 ```Order``` 컨트랙 생성에 필요한 gas 출력

```Order``` 컨트랙은 바이트코드량과 그 기능이 많아 배포하기 위해서는 gas가 많이 필요하다.



#### 1-3 고객정보 3건 입력

* 주소1을 사용하여 고객정보 입력 -> 111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul"

* 주소2를 사용하여 고객정보 입력 -> 112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul"

* 주소3을 사용하여 고객정보 입력 -> 113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul"

* 중복된 ID, 예를 들어 111, "kim", "010-2017-1111", "7 hongji-dong jongro-gu seoul"는 입력이 불가능하고, 오류 발생해야 한다.

또한 중복ID를 허용하지 않으므로 테스트하면서 고객정보를 여러 번 입력하려면 오류가 발생하게 된다. 이 경우 팁은 배포를 새로 하고, 고객정보를 입력하면 중복ID가 해소되어 가능하다는 점에 주의하자.

* 아래 ```tx.origin```와 ```msg.sender``` 관련 주의를 참조해서 ```msg.sender``` 대신 ```tx.orgin```을 사용하자.



#### 1-4 모든 고객의 배송지 출력

* 주소1을 사용하여 고객배송지 출력, "111 hongji-dong jongro-gu seoul"

* 주소2를사용하여 고객배송지 출력, "112 hongji-dong jongro-gu seoul"

* 주소3을 사용하여 고객배송지 출력, "113 hongji-dong jongro-gu seoul"

* 주소를 출력할 때, 다른 주소키를 사용한다면, ```call({from: accounts[1]})```과 같이 주소키를 적어주고 호출한다.



#### 1-5 주문

* 주문하면서, 주문금액을 전송해야 하고, 주문금액의 1% 마일리지를 주문자에게 부여

* 주문하면서, 주문상태를 "Ordered"로 설정하고 시간은 현재시간을 넣는다.

* 주소1에서 주문내역 입력 -> 555, "T-Shirt", 2, 1115

* 주소2에서 주문내역 입력 -> 556, "T-Shirt", 3, 1116

* 주소3에서 주문내역 입력 -> 557, "T-Shirt", 4, 1117



#### 1-6 주문개수, 주문금액 합계 및 잔고 출력

* 주문입력을 잘 했으면, 주문개수 3건, 주문금액 합계 3348, 컨트랙잔고를 출력한다.

* 잘 풀었으면 컨트랙잔고는 주문총액과 동일하다.

* 한글로 출력하지 않아도 된다.



#### 1-7 모든 고객의 주문 내역 출력

* 주소1에서 주문내역 출력 -> 555, "T-Shirt", "Ordered", "111 hongji-dong jongro-gu seoul"

* 주소2에서 주문내역 출력 -> 556, "T-Shirt", "Ordered", "112 hongji-dong jongro-gu seoul"

* 주소3에서 주문내역 출력 -> 557, "T-Shirt", "Ordered", "113 hongji-dong jongro-gu seoul"



#### 1-8 주문ID 556으로 주문내역 출력

* 주문ID 556으로 주문내역 출력 -> 556, "T-Shirt", "Ordered", "112 hongji-dong jongro-gu seoul"

* 주소키를 입력해서 출력하지 않는다.



#### 1-9 주문ID 556으로 환불처리

* 주문ID 556으로 환불처리, 단 주문ID 556을 생성한 동일한 주문ID로 해야 처리된다.

* 환불 후, 상품ID에 대해 주문상태를 "refunded"로, 주문금액도 0으로 설정하고 그에 따라 컨트랙 잔고도 감해준다.

* 주소2 잔고변동 출력. 주문ID 556은 주소2에서 발생했다. 주소2는 환불을 받았으니 1116이 환입되어 변동이 없어야 한다 (gas 비용 증감은 있을 수 있다.)



#### 1-10 환불처리가 반영된 출력

* 주문ID 556으로 주문내역 출력 -> 556, "T-Shirt", 0, "refunded", "112 hongji-dong jongro-gu seoul"

* 잘 풀었으면, 주문개수 2건, 주문금액 2232, 컨트랙 잔고 출력 (주문금액 합계와 동일한 잔고)



### 주의

거래가 호출자U1 -> 컨트랙C1 -> 컨트랙C2의 순서대로 완성이 될 경우:

* C2에서 ```msg.sender```는 바로 직전의 호출자C1을 말한다.

* 반면에 ```tx.origin```은 U1의 주소를 말한다.



아래 프로그램을 보자. 주소 0x69e9a...0c102에서 ```Order```의 ```getTxOriginMsgSender()``` 함수를 호출하면:

* ```Customer```의 ```tx.origin``` 은 0x69e9a...0c102 (```Order``` 컨트랙 함수호출자인 ```msg.sender```의 주소와 동일)

* ```Customer```의 ```msg.sender```는 0x0b878...E7cD4, 즉 ```Order``` 컨트랙 배포주소와 동일하다.



```python

pragma solidity ^0.6.0;
contract Customer {
    function getTxOriginMsgSender() view public returns(address, address) {
        return(tx.origin, msg.sender);
    }
}
contract Order {
    Customer c;
    constructor() public {
        c = new Customer();
    }

    function getTxOriginMsgSender() view public returns(address, address) {
        return c.getTxOriginMsgSender();
    }
}

```

In [64]:
%%writefile src/CustomerOrder.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.1;

contract Customer {
    struct userInfo {
        uint userId;
        string name;
        string phone;
        string addr;
        bool isCustomer;
    }
    
    mapping(address => userInfo) userMap;
    mapping(address => uint) mileage;
    mapping(address => uint) balanceOf;
    mapping(uint => address) addressById;

    address owner;

    constructor() {
        owner = tx.origin;
    }
    
    event PrintLog(string);

    function signUp(address addr, uint id, string memory name, string memory phone, string memory home) public {
        if(userMap[addressById[id]].isCustomer){
            emit PrintLog("Sorry the account already exist");
        }
        else{
            userInfo memory m=userInfo(id, name, phone, home, true);
            userMap[addr]=m;
            balanceOf[addr]=addr.balance;
            addressById[id]=addr;
        }
    }

    function addMileage(address addr, uint num) public {
        mileage[addr] += num;
    }

    function refundMileage(address addr, uint num) public {
        mileage[addr] -= num;
    }

    function withdrawMoney(address addr, uint num) public payable {
        balanceOf[addr] -= num;
    }        

    function refundMoney(address addr, uint num) public payable {
        balanceOf[addr] += num;
    }

    function printUserByAddress(address addr) view public onlyManager 
        returns(uint, string memory, string memory, string memory, uint) {
            userInfo memory u = userMap[addr];
            return(u.userId, u.name, u.phone, u.addr, mileage[addr]);
    }

    function returnHouseAddr(address addr) view public onlyManager returns(string memory) {
        userInfo memory u = userMap[addr];
        return u.addr;
    }

    function checkBalance(address addr) public view returns(uint) {
        return balanceOf[addr];
    }

    modifier onlyManager() {
        require(tx.origin == owner);
        _;
    }

}

contract Order {
    struct orderInfo {
        uint orderId;
        string productName;
        uint productCount;
        string userAddr;
        uint256 orderTime;
    }

    mapping(address => orderInfo) orderMap;
    mapping(address => uint) productCost;
    mapping(address => string) status;
    mapping(uint => address) addressByOrderId;
    mapping(address => uint) ownerBalance;

    address owner;
    Customer c;
    uint public orderCount = 0;
    
    constructor() {
        c = new Customer();
        owner = msg.sender;
    }

    function signUp(address addr, uint id, string memory name, string memory phone, string memory home) public {
        c.signUp(addr, id, name, phone, home);
    }

    function addMileage(address addr, uint num) public {
        c.addMileage(addr, num);
    }

    function refundMileage(address addr, uint num) public {
        c.refundMileage(addr, num);
    }

    function withdrawMoney(address addr, uint num) public payable {
        c.withdrawMoney(addr, num);
    }

    function refundMoney(address addr, uint num) public payable {
        c.refundMoney(addr, num);
    }

    function printUserByAddress(address addr) view public returns(uint, string memory, string memory, string memory, uint) {
        return c.printUserByAddress(addr);
    }

    function returnHouseAddr(address addr) view public returns(string memory) {
        return c.returnHouseAddr(addr);
    }

    function makeOrder(address addr, uint id, string memory pdname, uint pdcount, uint pdcost) public payable {
        orderInfo memory m = orderInfo(id, pdname, pdcount, returnHouseAddr(addr), block.timestamp);
        orderMap[addr] = m;
        addressByOrderId[id] = addr;
        productCost[addr] = pdcost;

        addMileage(addr, pdcost / 100);
        status[addr] = "ordered";
        orderCount += 1;
        
        require(msg.value == pdcost);
        withdrawMoney(addr, pdcost);
        ownerBalance[owner] += pdcost;
    }

    function printOrderByAddress(address addr) view public onlyManager 
        returns(uint, string memory, uint, string memory, string memory) {
        orderInfo memory u = orderMap[addr];
        return(u.orderId, u.productName, productCost[addr], status[addr], returnHouseAddr(addr));
    }

    function printOrderById(uint _id) view public onlyManager returns(uint, string memory, uint, string memory, string memory) {
        orderInfo memory u = orderMap[addressByOrderId[_id]];
        return(u.orderId, u.productName, productCost[addressByOrderId[_id]], status[addressByOrderId[_id]], returnHouseAddr(addressByOrderId[_id]));
    }

    function printOrderList() public view returns(uint, uint, uint) {
        return(orderCount, ownerBalance[owner], checkContractBalance());
    }

    function refund(uint _id) public payable {
        require(addressByOrderId[_id] == msg.sender);
        status[addressByOrderId[_id]] = "refunded";
        refundMileage(addressByOrderId[_id], productCost[addressByOrderId[_id]] / 100);
        refundMoney(addressByOrderId[_id], productCost[addressByOrderId[_id]]);
        ownerBalance[owner] -= productCost[addressByOrderId[_id]];
        payable(owner).transfer(productCost[addressByOrderId[_id]]);
        productCost[addressByOrderId[_id]] = 0;
        orderCount -= 1;
    }

    function checkBalance(address addr) view public returns(uint) {
        return c.checkBalance(addr);
    }

    function checkOwnerBalance() view public onlyManager returns(uint) {
        return ownerBalance[owner];
    }

    function checkContractBalance() view public onlyManager returns(uint) {
        return address(this).balance;
    }

    modifier onlyManager {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/CustomerOrder.sol


In [65]:
!solc-windows.exe --optimize --combined-json abi,bin src/CustomerOrder.sol > src/CustomerOrder.json

In [66]:
%%writefile src/CustomerOrderDeploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);
var _abiArray=_json.contracts["src/CustomerOrder.sol:Order"].abi;
var _bin="0x"+_json.contracts["src/CustomerOrder.sol:Order"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .estimateGas(function(err, gas) {
            if(!err) {
                console.log("1-(2) gas " + gas);
            }
        });
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 6588068})
        .on('transactionHash', function(hash) {
            console.log(">>> transactionHash : " + hash);
        })
        .on('receipt', function(receipt) {
            console.log(">>> receipt : " + receipt.transactionHash + "\n>>> address : " + receipt.contractAddress);
        }) 
        .on('error', function(error, receipt) {
            console.log(">>> error : " + error);
        });
    console.log("---> the contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/CustomerOrderDeploy.js


In [67]:
!node src/CustomerOrderDeploy.js

Deploying the contract from : 0x51967008fA944866Bb6Df61d1941E11760f33326
1-(2) gas 1924674
>>> transactionHash : 0xce47732d648f1889d99e49c2120060023915688e335371c1933644b39eb78dc0
>>> receipt : 0xce47732d648f1889d99e49c2120060023915688e335371c1933644b39eb78dc0
>>> address : 0x7BE35348BaBBd79039EF77cEe6f33FF28d24c2b2
---> the contract deployed to : 0x7BE35348BaBBd79039EF77cEe6f33FF28d24c2b2


In [68]:
%%writefile src/CustomerOrderUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);
var _abiArray=_json.contracts["src/CustomerOrder.sol:Order"].abi;
var _bin="0x"+_json.contracts["src/CustomerOrder.sol:Order"].bin;

var order = new web3.eth.Contract(_abiArray, "0x7BE35348BaBBd79039EF77cEe6f33FF28d24c2b2");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    // 1-(1)
    console.log("1-(1)");
    console.log("account[1] : " + accounts[1] + " balance of account[1] : " + await web3.eth.getBalance(accounts[1]));
    console.log("account[2] : " + accounts[2] + " balance of account[2] : " + await web3.eth.getBalance(accounts[2]));
    console.log("account[3] : " + accounts[3] + " balance of account[3] : " + await web3.eth.getBalance(accounts[3]));
    
    web3.eth.getBlockNumber(function(error, result) {
        if(!error) {
            console.log("block number : " + result);
        }
    });
    
    // 1-(3)
    await order.methods.signUp(accounts[1], 111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul")
        .send({from: accounts[0], gas: 387502});
    await order.methods.signUp(accounts[2], 112, "lee", "010-2017-1112", "111 hongji-dong jongro-gu seoul")
        .send({from: accounts[0], gas: 387502});
    await order.methods.signUp(accounts[1], 113, "lim", "010-2017-1113", "111 hongji-dong jongro-gu seoul")
        .send({from: accounts[0], gas: 387502});
    
    // 1-(4)
    console.log("1-(4)");
    order.methods.returnHouseAddr(accounts[1])
        .call()
        .then(function(value) {
            console.log("account[1] house addr : " + value);
        });
    order.methods.returnHouseAddr(accounts[2])
        .call()
        .then(function(value) {
            console.log("account[2] house addr : " + value);
        });
    order.methods.returnHouseAddr(accounts[3])
        .call()
        .then(function(value) {
            console.log("account[3] house addr : " + value);
        });
    
    // 1-(5)
    await order.methods.makeOrder(accounts[1], 555, "T-Shirt", 2, 1115)
        .send({from: accounts[0], gas: 387502, value: 1115});
    await order.methods.makeOrder(accounts[2], 556, "T-Shirt", 3, 1116)
        .send({from: accounts[0], gas: 387502, value: 1116});
    await order.methods.makeOrder(accounts[3], 557, "T-Shirt", 4, 1117)
        .send({from: accounts[0], gas: 387502, value: 1117});
    order.methods.printUserByAddress(accounts[1])
        .call()
        .then(function(value) {
            console.log("1-(5)\n", "user id : ", value[0], ", user name : ", value[1], ", mileage : ", value[4]);
        });
    order.methods.printUserByAddress(accounts[2])
        .call()
        .then(function(value) {
            console.log("user id : ", value[0], ", user name : ", value[1], ", mileage : ", value[4]);
        });
    order.methods.printUserByAddress(accounts[3])
        .call()
        .then(function(value) {
            console.log("user id : ", value[0], ", user name : ", value[1], ", mileage : ", value[4]);
        });
    
    // 1-(6)
    order.methods.printOrderList()
        .call()
        .then(function(value) {
            console.log("1-(6)\ncounted order time : ", value[0], ", total amount : ", value[1], ", contract balance : ", value[2]);
        });
    
    // 1-(7)
    order.methods.printOrderByAddress(accounts[1])
        .call()
        .then(function(value) {
            console.log("1-(7)\norder id : ", value[0], ", product : ", value[1], ", order amount : ", value[2], ", status : ", value[3], ", house addr : ", value[4]);
        });
    order.methods.printOrderByAddress(accounts[2])
        .call()
        .then(function(value) {
            console.log("order id : ", value[0], ", product : ", value[1], ", order amount : ", value[2], ", status : ", value[3], ", house addr : ", value[4]);
        });
    order.methods.printOrderByAddress(accounts[3])
        .call()
        .then(function(value) {
            console.log("order id : ", value[0], ", product : ", value[1], ", order amount : ", value[2], ", status : ", value[3], ", house addr : ", value[4]);
        });
    
    // 1-(8)
    order.methods.printOrderById(556)
        .call()
        .then(function(value) {
            console.log("1-(8)\norder id : ", value[0], ", product : ", value[1], ", order amount : ", value[2], ", status : ", value[3], ", house addr : ", value[4]);
        });
    
    // 1-(9)
    order.methods.checkBalance(accounts[2])
        .call()
        .then(function(value) {
            console.log("1-(9)\nbefore refund : ", value);
        });
    await order.methods.refund(556).send({from: accounts[2], gas: 387502});
    order.methods.checkBalance(accounts[2])
        .call()
        .then(function(value) {
            console.log("after refund : ", value);
        });
    
    // 1-(10)
    order.methods.printOrderById(556)
        .call()
        .then(function(value) {
            console.log("1-(10)\norder id : ", value[0], ", product : ", value[1], ", order amount : ", value[2], ", status : ", value[3], ", house addr : ", value[4]);
        });
    order.methods.printOrderList()
        .call()
        .then(function(value) {
            console.log("counted order time : ", value[0], ", total amount : ", value[1], ", contract balance : ", value[2]);
        });
    order.methods.printUserByAddress(accounts[2])
        .call()
        .then(function(value) {
            console.log("account[2] milage >>> user id : ", value[0], ", user name : ", value[1], ", milegae : ", value[4]);
        })
}

doIt();

Overwriting src/CustomerOrderUse.js


In [69]:
!node src/CustomerOrderUse.js

1-(1)

C:\Users\youve\Code\201810820\node_modules\web3-core-helpers\src\errors.js:29
        return new Error('Returned error: ' + message);
               ^

Error: Returned error: VM Exception while processing transaction: revert
    at Object.ErrorResponse (C:\Users\youve\Code\201810820\node_modules\web3-core-helpers\src\errors.js:29:16)
    at C:\Users\youve\Code\201810820\node_modules\web3-core-requestmanager\src\index.js:140:36
    at XMLHttpRequest.request.onreadystatechange (C:\Users\youve\Code\201810820\node_modules\web3-providers-http\src\index.js:110:13)
    at XMLHttpRequestEventTarget.dispatchEvent (C:\Users\youve\Code\201810820\node_modules\xhr2-cookies\dist\xml-http-request-event-target.js:34:22)
    at XMLHttpRequest._setReadyState (C:\Users\youve\Code\201810820\node_modules\xhr2-cookies\dist\xml-http-request.js:208:14)
    at XMLHttpRequest._onHttpResponseEnd (C:\Users\youve\Code\201810820\node_modules\xhr2-cookies\dist\xml-http-request.js:318:14)
    at IncomingMessage.<anon


account[1] : 0xe46df1Fe05F21439050CAdf05A494f97a5AD2F24 balance of account[1] : 1000000000000000000000
account[2] : 0xc3D31b7C15F40fE60cE8ef21Aaf2F046976c0f1b balance of account[2] : 1000000000000000000000
account[3] : 0x4Bf831e788D267B4b5A3408F2C2841AE7f9140de balance of account[3] : 1000000000000000000000
block number : 43
1-(4)
